In [ ]:
%load_ext autoreload
%autoreload 2

import os

CURRENT_DPATH = os.path.abspath(os.path.dirname("__file__"))
PROJECT_ROOT = os.path.abspath(os.path.join(CURRENT_DPATH, os.pardir, os.pardir))
DATA_DPATH = os.path.join(PROJECT_ROOT, "data")

import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd 
from ipywidgets import interact, IntSlider
from tqdm import tqdm

from lane_detection_hackathon.utils.fs import read_image
from lane_detection_hackathon.datasets import SegmentationDataset

## Data Loading

In [ ]:
prep_dpath = os.path.join(DATA_DPATH, "apolloscapes_preprocessed")

In [ ]:
fpath = os.path.join(prep_dpath, "2023_02_26", "raw_data.xlsx")
raw_df = pd.read_excel(fpath, index_col=0)

raw_df.shape

## Dataset Debugging

In [ ]:
label_map = {
    "background": 0,
    "SYD": 1,
    "BWG": 2,
    "SWD": 3,
    "SWS": 4,
    "CWYZ": 5
}

dataset = SegmentationDataset(prep_dpath, raw_df, label_map)

In [ ]:
@interact
def show(index=IntSlider(value=0, min=0, max=len(dataset) - 1)):
    img_t, masks_t, _weights = dataset[index]

    img_np = img_t.permute(1, 2, 0).cpu().detach().numpy()

    masks_np = masks_t.permute(1, 2, 0).cpu().detach().numpy()

    mask_fpath = os.path.join(prep_dpath, raw_df.loc[index]["tgt"])
    src_mask = read_image(mask_fpath)

    _fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

    ax[0].imshow(img_np)
    ax[1].imshow(src_mask)
    plt.show() 

    _fig, ax = plt.subplots(nrows=1, ncols=len(label_map), figsize=(20, 6))

    for ind, label_name in enumerate(label_map.keys()):
        ax[ind].imshow(masks_np[:, :, ind], cmap="gray", vmin=0, vmax=1)
        ax[ind].set_title(label_name)

    plt.show()